### logging

In [ ]:
if globals().get('LOGGING_LEVEL') == None:
    # LOGGING_LEVEL = 0 # no logging
    # LOGGING_LEVEL = 1 # function entry logging
    LOGGING_LEVEL = 2 # detailed logging

### modules

In [ ]:
if globals().get('LOADED_ANNOTATIONS') != True:
    %run ANNOTATIONS.ipynb

In [ ]:
if globals().get('LOADED_PATTERN_OBSERVER') == None:
    %run PATTERN_OBSERVER.ipynb

In [ ]:
if globals().get('LOADED_CHECKBOXES') == None:
    %run CHECKBOXES.ipynb

In [ ]:
if globals().get('LOADED_DATA_CONTAINER') == None:
    %run DATA_CONTAINER.ipynb

In [ ]:
if globals().get('LOADED_SETTINGS') == None:
    %run SETTINGS.ipynb

In [ ]:
if globals().get('LOADED_VIEWING_TABS_EXTRADAY_ANALYSIS') == None:
    %run VIEWING_TABS_EXTRADAY_ANALYSIS.ipynb

In [ ]:
if globals().get('LOADED_VIEWING_TABS_INTRADAY_ANALYSIS') == None:
    %run VIEWING_TABS_INTRADAY_ANALYSIS.ipynb

In [ ]:
if globals().get('LOADED_VIEWING_TABS_FACTOR_ANALYSIS') == None:
    %run VIEWING_TABS_FACTOR_ANALYSIS.ipynb

### imports

In [ ]:
from ipywidgets import HTML

### begin

In [ ]:
class BACKTEST_VIEWER(IObserver):
    
    _tab_collection_dico = {
        'Extraday' : 'EXTRADAY_TAB_COLLECTION()',
        'Intraday' : 'INTRADAY_TAB_COLLECTION()',
        'Factor'   : 'FACTOR_TAB_COLLECTION()',
    }

    def __init__(self,
        name='BACKTEST_VIEWER'
        ):
        
        self._logging_level                 = LOGGING_LEVEL                  # GLOBAL VARIABLE

        self.name                           = name
        self._observers                     = [] # for observer pattern
        
        ####################################
        # instantiate SETTINGS
        ####################################
        # needs
        # - nothing
        # with / gives                 SETTINGS               BACKTEST_SELECTOR      BOOK_SELECTOR          DATA_CONTAINER         BACKTEST_VIEWER        TAB_COLLECTION
        # - SETTINGS[reload_data]                                                                           [repopulate data_dico]
        # - SETTINGS[backtest_path]                           [compute backtests]
        # - SETTINGS[return_type]                                                                                                  .update plot device.
        # - SETTINGS[plot_method]                                                                                                  [update plot device]
        # - SETTINGS[view_type]                                                                             [load correct data]    [update plot device]
        # - SETTINGS[date filter]                                                                           [filter data]
        # - SETTINGS[time filter]                                                                           [filter data]
        ####################################
        self._SETTINGS = SETTINGS(
            name                            = 'SETTINGS',
            width                           = '20%',
        )

        ####################################
        # instantiate BACKTEST_SELECTOR
        ####################################
        # needs
        # - SETTINGS[backtests]
        # with / gives                 SETTINGS               BACKTEST_SELECTOR      BOOK_SELECTOR          DATA_CONTAINER         BACKTEST_VIEWER        TAB_COLLECTION
        # - SETTINGS[backtests]                                                      .compute books.
        ####################################
        self._BACKTEST_SELECTOR = CHECKBOXES(
            name                            = 'BACKTEST_SELECTOR',
            options                         = self._settings_all_backtests,
            default                         = False,
            width                           = '60%',
        )
            
        ####################################
        # instantiate BOOK_SELECTOR
        ####################################
        # needs
        # - BACKTEST_SELECTOR[books]
        # with / gives                 SETTINGS               BACKTEST_SELECTOR      BOOK_SELECTOR          DATA_CONTAINER         BACKTEST_VIEWER        TAB_COLLECTION
        # - BACKTEST_SELECTOR[books]                                                                        grab correct books
        ####################################
        self._BOOK_SELECTOR = CHECKBOXES(
            name                            = 'BOOK_SELECTOR',
            options                         = self._settings_all_books, # TODO `BACKTEST SELECTOR` needs to have `all_books`
            default                         = True,
            width                           = '20%',
        )

        ####################################
        # instantiate BACKTEST_VIEWER
        ####################################
        # needs
        # - SETTINGS[view_type]
        # with / gives                 SETTINGS               BACKTEST_SELECTOR      BOOK_SELECTOR          DATA_CONTAINER         BACKTEST_VIEWER        TAB_COLLECTION
        # - SETTINGS[view_type]                                                                                                    update plot device
        # - SETTINGS[plot_method]                                                                                                  update plot device
        # - SETTINGS[return_type]                                                                                                  update plot device
        ####################################

        ####################################
        # instantiate DATA_CONTAINER
        ####################################
        # needs
        # - view_type
        # - selected backtests
        # - books
        # - date / time filter params
        # with / gives                 SETTINGS               BACKTEST_SELECTOR      BOOK_SELECTOR          DATA_CONTAINER         BACKTEST_VIEWER        TAB_COLLECTION
        # - creates df for plotter (TAB_COLLECTION)
        ####################################
        self._DATA_CONTAINER = DATA_CONTAINER( # TODO needs to observe changes in `self.selected_backtests`, `self.selected_books` and `self.applied_settings`
            
            # options
            selected_backtests    = self._selected_backtests,
            selected_books        = self._selected_books,

            # reference
            plot_method           = self._settings_plot_method,
            return_type           = self._settings_return_type,
            view_type             = self._settings_view_type,
            date_from             = self._settings_date_from,
            date_to               = self._settings_date_to,
            time_from             = self._settings_date_from,
            time_to               = self._settings_date_to,
        )
        
        ####################################
        # build GUI
        ####################################
        self.reference            = {}
        self.widget               = self._make_widget(self._settings_view_type) # TODO needs to observe changes in self._SETTINGS

        ####################################
        # class linking
        ####################################
        self._SETTINGS.attach(self)                           # self._BACKTEST_VIEWER observes self._SETTINGS and coordinates messages received from it
        self._BACKTEST_SELECTOR.attach(self)                  # self._BACKTEST_VIEWER observes self._BACKTEST_SELECTOR and updates self._DATA_CONTAINER off it
        self._BOOK_SELECTOR.attach(self)                      # self._BACKTEST_VIEWER observes self._BOOK_SELECTOR and updates self._DATA_CONTAINER off it

    ####################################
    # observer pattern
    ####################################
    # observer
    def react(self,
        subject_name : str,
        subject_info : object
        ) -> None :
        if self._logging_level > 0: print('OBSERVER PATTERN',':',self.name,'REACTS','subject_name',subject_name)
        if self._logging_level > 0: print('OBSERVER PATTERN',':',self.name,'REACTS','subject_info',subject_info.keys())

        # default to no need to rebuild df
        build_df       = False
        recompute_rtns = False
        redraw_plots   = False
        
        ###################
        # SETTINGS notified
        ###################
        if subject_name == 'SETTINGS':
            if self._logging_level > 0: print(self.name,':','react',subject_info)
            
            #=======
            # for self._BACKTEST_SELECTOR | self._BOOK_SELECTOR
            #=======
            if ('all_backtests' in subject_info):
                # backtest path changed, must strip and rebuild
                build_df       |= True
                recompute_rtns |= True
                redraw_plots   |= True
                
                if self._logging_level > 1: print(self.name,':','all_backtests')
                
                # update self._BACKTEST_SELECTOR options
                self._BACKTEST_SELECTOR.update_options(subject_info['all_backtests'])
                self._DATA_CONTAINER.reference['selected_backtests'] = self._BACKTEST_SELECTOR.applied_settings # tell self._DATA_CONTAINER backtests have changed
                
                # update self._BOOK_SELECTOR options
                all_books=[] # identify all books TODO
                self._BOOK_SELECTOR.update_options(subject_info['all_books']) # TODO, need to compute this from family of backtests
                self._DATA_CONTAINER.reference['selected_books']     = self._BOOK_SELECTOR.applied_settings     # tell self._DATA_CONTAINER books have changed
                
            if ('app_name' in subject_info):
                # app_name changed
                build_df       |= False
                recompute_rtns |= False
                redraw_plots   |= True
                
                self._header_widget.value=self._header(subject_info['app_name'])
                
            #=======
            # for self._DATA_CONTAINER
            #=======
            if ('view_type' in subject_info):
                # view_type changed, must rebuild df as data different + update plotting device
                if self._logging_level > 1: print(self.name,':','view_type',subject_info)
                build_df       |= True
                recompute_rtns |= False
                redraw_plots   |= True

                # self._DATA_CONTAINER needs to load different files
                self._DATA_CONTAINER.reference['view_type']    = subject_info['view_type']
                
                # plotting tool needs to change how it looks
                self._TAB_COLLECTION = self._make_tab_collection(subject_info['view_type']) # make new tab collection
                self._tab_widgets = self._TAB_COLLECTION.widget.children                    # update frontend

            # min-date changed
            if ('date_from' in subject_info):
                if self._logging_level > 0: print(self.name,':','update[date_from]')
                self._DATA_CONTAINER.reference['date_from']    = subject_info['date_from']
                build_df       |= True
                recompute_rtns |= True
                redraw_plots   |= True
            
            # max-date changed
            if ('date_to' in subject_info):
                if self._logging_level > 0: print(self.name,':','update[date_to]')
                self._DATA_CONTAINER.reference['date_to']      = subject_info['date_to']
                build_df       |= True
                recompute_rtns |= True
                redraw_plots   |= True
            
            # min-time changed
            if ('time_from' in subject_info) & (self._DATA_CONTAINER.reference['view_type'] == 'Intraday'):
                if self._logging_level > 0: print(self.name,':','update[time_from]')
                self._DATA_CONTAINER.reference['time_from']    = subject_info['time_from']
                build_df       |= True
                recompute_rtns |= True
                redraw_plots   |= True
            
            # max-time changed
            if ('time_to' in subject_info) & (self._DATA_CONTAINER.reference['view_type'] == 'Intraday'):
                if self._logging_level > 0: print(self.name,':','update[time_to]')
                self._DATA_CONTAINER.reference['time_to']      = subject_info['time_to']
                build_df       |= True
                recompute_rtns |= True
                redraw_plots   |= True
            
            # asked to repopulate data
            if ('reload_data' in subject_info):
                if self._logging_level > 0: print(self.name,':','clear data_dico')
                self._DATA_CONTAINER.data_dico.clear()
                build_df       |= True
                recompute_rtns |= True
                redraw_plots   |= True

            #=======
            # for self._TAB_COLLECTION
            #=======
            if ('plot_method' in subject_info):
                self._DATA_CONTAINER.reference['plot_method'] = subject_info['plot_method']
                # plotting method changed (doesn't impact df), no need to flag rebuild
                build_df       |= False
                recompute_rtns |= False
                redraw_plots   |= True

            if ('return_type' in subject_info):
                self._DATA_CONTAINER.reference['return_type'] = subject_info['return_type']
                # plotting method changed (doesn't impact df), no need to flag rebuild
                build_df       |= False
                recompute_rtns |= False
                redraw_plots   |= True

        ###################
        # BACKTEST_SELECTOR notified
        ###################
        if subject_name == 'BACKTEST_SELECTOR':
            # backtest selection changed
            if self._logging_level > 0: print(self.name,':','backtests changed')
            self._DATA_CONTAINER.reference['selected_backtests'] = subject_info
            build_df       |= True
            recompute_rtns |= True
            redraw_plots   |= True

        ###################
        # BOOK_SELECTOR notified
        ###################
        if subject_name == 'BOOK_SELECTOR':
            # book selection changed
            if self._logging_level > 0: print(self.name,':','books changed')
            self._DATA_CONTAINER.reference['selected_books'] = subject_info
            build_df       |= True
            recompute_rtns |= True
            redraw_plots   |= True

        ###################
        # rebuild df as needed
        ###################
        if build_df:
            if self._logging_level > 0: print(self.name,':','build_df')
            self._DATA_CONTAINER.build_df()

        ###################
        # redraw plots as needed
        ###################
        if redraw_plots:
            if self._logging_level > 0: print(self.name,':','redraw_plots')
            self._TAB_COLLECTION.update_collection(
                self._DATA_CONTAINER.df,
                self._DATA_CONTAINER.reference['plot_method'],
            )

        ###################
        # recompute returns as needed
        ###################
        if recompute_rtns:
            if self._logging_level > 0: print(self.name,':','recompute_rtns')
            # TODO
        
    def _header(self,txt,fontsize=50):
        return "<h1 style='font-size:"+str(fontsize)+"px'>"+txt+"</h1>"

    ####################################
    # build GUI
    ####################################
    def _make_widget(self,view_type : str) -> VBox :
        
        # header
        header = HTML(self._header(self._SETTINGS.reference['app_name']))

        # controls
        controls = HBox(
            [
                self._BACKTEST_SELECTOR.widget,
                self._BOOK_SELECTOR.widget,
                self._SETTINGS.widget,
            ]
        )
        
        # make viewing tabs
        self._TAB_COLLECTION = self._make_tab_collection(view_type)

        # build screen
        return VBox(
            [
                header,
                controls,
                self._TAB_COLLECTION.widget
            ]
        )
    
    def _make_tab_collection(self,view_type : str) -> TAB_COLLECTION :
        # update reference
        self.reference['applied_view_type'] = view_type
        
        # instantiate
        func = BACKTEST_VIEWER._tab_collection_dico[view_type]
        
        # debugging
        if self._logging_level > 0: print(self.name,':','_make_tab_collection',view_type,func)
        
        # make and return
        return eval(func)

    ####################################
    # properties
    ####################################
    # from self._SETTINGS
    @property
    def _settings_backtest_path(self) -> str :
        return self._SETTINGS.reference['backtest_path']

    @property
    def _settings_plot_method(self) -> str :
        return self._SETTINGS.reference['plot_method']

    @property
    def _settings_return_type(self) -> str :
        return self._SETTINGS.reference['return_type']

    @property
    def _settings_view_type(self) -> str :
        return self._SETTINGS.reference['view_type']

    @property
    def _settings_date_from(self) -> object :
        return self._SETTINGS.reference['date_from']

    @property
    def _settings_date_to(self) -> object :
        return self._SETTINGS.reference['date_to']

    @property
    def _settings_all_backtests(self) -> ID_PATHS :
        return self._SETTINGS.reference['all_backtests']

    @property
    def _settings_all_books(self) -> ID_PATHS :
        return self._SETTINGS.reference['all_books']

    # from self._BACKTEST_SELECTOR
    @property
    def _selected_backtests(self) -> OPTIONID_VALUES:
        return self._BACKTEST_SELECTOR.applied_settings
        
    # from self._BOOK_SELECTOR
    @property
    def _selected_books(self) -> OPTIONID_VALUES:
        return self._BOOK_SELECTOR.applied_settings

    # from self.widget
    @property
    def _header_widget(self) -> HTML :
        return self.widget.children[0]
    
    @property
    def _control_widgets(self) -> BUTTONS :
        return self.widget.children[1].children
    
    @property
    def _tab_widgets(self) -> TAB_COLLECTION :
        return self.widget.children[2].children

    @_tab_widgets.setter
    def _tab_widgets(self,new_tab_collection : TAB_COLLECTION) -> None:
        self.widget.children[2].children = new_tab_collection

In [ ]:
# __file__ exists if notebook called with %run but doesnt it called manually
# e.g. I only wish to run the example when calling this notebook directly
try:
    __file__
except NameError:
    # example
    a=BACKTEST_VIEWER()
    display(dir(a))
    display(a.widget)

debugging
- from IPython.core.debugger import set_trade

ideas
- days of the week plot